### Energy-based Flow Classifier (EFC)

In [1]:
# EFC requires discretized data as input


In [1]:
%run efc/usage_example.py

Single-class results
[[  7  36]
 [  0 145]]


/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/efc/generic_discretize.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data.iloc[:,feature].fillna(len(intervals[feature]), inplace=True)
/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/efc/generic_discretize.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work

Multi-class results
[[ 0  0 11]
 [ 0  0 48]
 [ 0  0  0]]


In [2]:
from efc.classification_functions import OneClassFit, OneClassPredict, MultiClassFit, MultiClassPredict

In [4]:
btc_trades_df_efc = prepare_dataset()

NameError: name 'prepare_dataset' is not defined

In [3]:
df_day_efc = btc_trades_df_efc.loc['2024-03-24':'2024-03-24'][['price', 'amount']]

NameError: name 'btc_trades_df_efc' is not defined